In [1]:
# *** GLOBAL IMPORT ***
import tensorflow as tf
from tf_yolo_utils import *   # fcts. to create custom layers

# Import kitti_utils from a different project
import sys
sys.path.insert(0, '/data2/lucas/Projects/Kitti2012')
from kitti_utils import *    # fcts. to manage the kitti dataset

# Hyperparameter Tuning

This notebook is here to explore the effect of the different hyperparameters in the YOLO NN.

## Grid Cells
In this part, we explore the effect of the number of grid cells in the picture.

In [3]:
# Import list of ids
ids_for_training = get_data_list('train')

In [4]:
# get the minimum number to have one object per cell-grid

S = 150     # Number of cell-grid size

im_sample = import_im(1, 'train')
im_height = im_sample.shape[0]
im_width = im_sample.shape[1]

general_max_obj_per_cell = 1
num_im_with_overlap = 0
list_num_obj_per_im = []

for im_id in ids_for_training:
# im_id = train_batches_ids[0][0]
    grid_cells = np.zeros((S,S))
    labels = import_labels(im_id, 'train')
    list_num_obj_per_im.append(len(labels)) 

    for label in labels:
        x_box_length = label['bbox']['x_max'] - label['bbox']['x_min']
        x_center = (label['bbox']['x_min'] + (x_box_length/2)) / im_width

        y_box_length = label['bbox']['y_max'] - label['bbox']['y_min']
        y_center = (label['bbox']['y_min'] + (y_box_length/2)) / im_height

        x_cell = math.floor(x_center * S)
        y_cell = math.floor(y_center * S)

        # print(label['type'] + ' with x_center: ' + str(x_center) + ' and y_center: ' + str(y_center) + ' is going to grid cell: (' + str(y_cell) + ', ' + str(x_cell) + ')')
        grid_cells[y_cell][x_cell] += 1 
    
    max_obj_per_cell = np.amax(grid_cells)
    
    if max_obj_per_cell > 1:
        num_im_with_overlap += 1
    
    if max_obj_per_cell > general_max_obj_per_cell:
        general_max_obj_per_cell = max_obj_per_cell
        
perc_im_with_overlap = num_im_with_overlap / len(ids_for_training) * 100

print('Percentage image where not all object are classify: ' + str(perc_im_with_overlap) + ' with max per cell = ' + str(general_max_obj_per_cell))
      

Percentage image where not all object are classify: 0.6015238604464644 with max per cell = 3.0
